In [4]:
import os 
import zipfile
import StringIO
import pandas as pd

datadir = '/Users/kanaan/Google Drive/TS-EUROTRAIN/RESULTS_QSMv3/SEPT10/AHBA/ahba_data/'

donors = ['donor1', 'donor2', 'donor3', 'donor4', 'donor5', 'donor6']

In [5]:
def return_donor_geneexpression(donor):
    
    #I/O
    donor_zip  = zipfile.ZipFile(os.path.join(datadir,'%s.zip'%donor))
    
    #read microarray data. take care indexing.... should have 58692 probes
    df_microarray = pd.read_csv(StringIO.StringIO(donor_zip.read('MicroarrayExpression.csv')),
                                header= None,index_col =0 )
    
    # read probe meta-data
    df_probes = pd.read_csv(StringIO.StringIO(donor_zip.read('Probes.csv')), index_col =0 )
    del df_probes.index.name
    
    # drop unannotated genes... should end up with 48170 probes
    drop_strings = ['uncharacterized', 'non-RefSeq']
    drop_strings = ['non-RefSeq']
    drop_probes  = [i for i in df_probes.index for x in drop_strings if x in df_probes.loc[i]['gene_name'] ]
    df_probes = df_probes.drop(drop_probes, axis =0)
    
    # Calc mean probe expression for each gene
    df_gene_exp = pd.concat([df_microarray, df_probes['gene_symbol']],axis=1).groupby('gene_symbol').mean().T.reset_index().drop(['index'],axis=1)
    df_gene_exp.columns.name = None
    
    # Add metadata to dataframe
    df_meta = pd.read_csv(StringIO.StringIO(donor_zip.read('SampleAnnot.csv')))
    df = pd.concat([df_gene_exp,df_meta],axis=1).set_index('well_id')
    del df.index.name
    
    # append donot name
    df['donor'] = donor
    return df


In [6]:
df_all = []
for donor in donors:
    df = return_donor_geneexpression(donor)
    df_all.append(df)

In [7]:
df_all[0]

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,...,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,donor
594,12.257452,4.307083,1.639045,6.047821,11.502535,2.992096,7.702302,4.657058,1.675294,3.470479,...,PCLa-i,"paracentral lobule, anterior part, right, infe...",37470,87,52,116,5.9,-27.7,49.7,donor1
2985,12.971966,5.394092,2.016619,6.258501,12.314500,3.450627,8.285363,5.044534,3.596557,4.651128,...,Cl,"claustrum, right",40517,66,92,63,29.2,17.0,-2.9,donor1
2801,13.397286,6.196477,2.419608,5.442959,12.776440,3.652065,8.698773,4.777458,3.662503,4.845015,...,Cl,"claustrum, right",41516,66,81,104,28.2,-22.8,16.8,donor1
2273,13.326607,5.665946,2.473735,4.363560,12.428296,4.512593,9.333068,5.325190,2.956374,3.377773,...,LGd,"dorsal lateral geniculate nucleus, left",41473,116,94,101,-24.6,-24.6,1.3,donor1
2785,13.356659,5.087975,2.391748,4.590116,13.365749,3.558010,9.939992,5.247008,3.760852,4.446860,...,CA4,"CA4 field, right",41142,63,104,106,31.1,-31.3,-7.3,donor1
2793,14.097546,5.008133,1.631623,6.620493,13.778565,2.772909,7.269222,5.572003,2.566058,2.372902,...,DG,"dentate gyrus, right",41141,63,104,107,31.1,-32.3,-7.1,donor1
2795,12.257307,5.771079,2.250821,6.101249,10.237291,4.092093,9.906851,5.482967,3.656102,4.070411,...,Dt,"dentate nucleus, left",73345,101,130,123,-9.7,-57.4,-32.6,donor1
2787,12.429456,5.773837,2.431591,4.686517,9.948550,3.902574,9.179340,5.207539,4.069870,4.138006,...,Fas,"fastigial nucleus, left",75936,90,127,125,1.9,-58.4,-28.7,donor1
2874,12.739914,6.181961,2.398844,6.204205,9.657435,3.787046,9.935296,4.731728,3.828880,4.654104,...,Emb,"emboliform nucleus, right",56000,84,125,118,8.5,-50.3,-27.8,donor1
2858,12.043521,5.013767,1.947309,6.272927,11.775498,2.128210,7.709195,4.939403,2.194948,4.038434,...,S,"subiculum, left",41212,114,108,100,-22.6,-27.4,-13.8,donor1


In [8]:
df_all[1]

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,...,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,donor
5705,12.233134,5.321921,2.166305,5.482180,11.819855,2.851921,8.042239,5.062174,2.075208,3.414446,...,Cl,"claustrum, left",978619,120,91,71,-29.2,5.8,-2.6,donor2
5713,12.316856,5.018191,1.961696,6.134849,12.436989,3.642371,8.557210,5.018321,1.279049,3.050612,...,Acb,"nucleus accumbens, left",998603,103,96,71,-10.1,5.9,-8.4,donor2
5721,12.374035,5.790579,1.333967,7.122039,12.883375,4.371786,9.890256,4.266128,2.316213,2.814452,...,Acb,"nucleus accumbens, right",998604,86,95,69,8.9,8.5,-7.4,donor2
5729,12.168797,5.152017,1.279049,5.969047,11.867710,3.600621,8.783679,4.262765,1.869106,3.151701,...,Acb,"nucleus accumbens, right",999200,87,95,76,7.9,0.8,-6.7,donor2
5737,11.663920,5.442288,1.618193,5.381421,11.892481,3.462606,8.473183,4.514223,1.279049,2.878600,...,SI,"substantia innominata, right",992030,79,97,76,16.9,0.9,-9.1,donor2
5745,12.447432,5.170098,1.740280,6.016476,12.582150,2.690884,8.124583,5.229579,2.470118,3.189743,...,Cl,"claustrum, right",992127,61,86,78,37.0,-0.5,4.1,donor2
5753,12.004784,5.731032,2.042131,5.381251,9.168999,3.839101,9.418330,4.612422,4.282830,2.959285,...,GPe,"globus pallidus, external segment, right",991186,82,89,75,13.4,2.2,0.2,donor2
5666,12.281470,5.822762,2.535768,5.630348,12.589563,3.817013,8.947572,5.563875,2.718003,2.266719,...,LA,"lateral nucleus, right",992123,72,108,75,24.8,1.7,-22.1,donor2
5674,11.977177,5.451586,1.861653,5.917503,11.720735,3.793267,8.514937,5.074579,1.685073,3.267146,...,ATZ,"amygdalohippocampal transition zone, right",992126,77,102,76,19.2,0.7,-14.9,donor2
5682,12.014082,5.638487,1.673395,5.387553,12.060680,3.281269,8.530788,4.645445,1.406957,3.029063,...,BLA,"basolateral nucleus, right",992121,78,108,75,18.1,1.6,-22.1,donor2


In [9]:
df_all[2]

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,...,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,donor
11281,12.889573,5.049439,2.042317,6.168989,12.040668,3.063961,8.695483,5.388024,3.393456,3.140641,...,MTG-i,"middle temporal gyrus, left, inferior bank of ...",1283581,149,106,137,-58.0,-46.0,3.0,donor3
11305,13.003047,5.173558,1.973284,6.154843,12.183389,2.988897,8.257742,6.092457,3.255138,2.862612,...,ITG-mts,"inferior temporal gyrus, left, bank of mts",1283601,152,128,137,-61.0,-46.0,-19.0,donor3
11289,12.998348,5.068065,1.996434,6.158318,12.233327,3.140788,8.227760,5.474782,2.792448,3.270367,...,ITG-its,"inferior temporal gyrus, left, bank of the its",1283603,143,126,137,-52.0,-46.0,-17.0,donor3
11335,13.128374,5.114931,1.981823,6.334744,12.369833,2.990184,8.547374,5.556616,2.997993,3.002738,...,MTG-s,"middle temporal gyrus, left, superior bank of ...",1279507,151,108,127,-60.0,-36.0,1.0,donor3
11319,12.633394,4.945027,2.108420,6.269155,12.110327,2.843943,7.657725,5.173323,1.969178,2.934619,...,STG-l,"superior temporal gyrus, left, lateral bank of...",1283331,160,100,127,-69.0,-36.0,9.0,donor3
11263,12.757907,5.265166,2.038109,6.099178,12.023067,3.274881,8.201581,5.542284,3.755017,2.820419,...,MTG-i,"middle temporal gyrus, left, inferior bank of ...",1284160,149,128,110,-58.0,-19.0,-19.0,donor3
11326,12.787183,5.103953,1.805359,6.349220,12.088231,2.840541,8.341424,5.377821,2.461249,3.311521,...,SIG,"short insular gyri, left",1280038,134,96,99,-43.0,-8.0,13.0,donor3
11318,12.905227,5.605071,2.067792,6.038727,11.871160,3.325424,8.096077,5.493541,3.398597,3.407218,...,LIG,"long insular gyri, left",1280037,132,104,100,-41.0,-9.0,5.0,donor3
11310,12.916432,5.100214,2.043607,6.418374,12.224190,2.830560,8.313791,6.059990,3.306293,3.458822,...,PLP,"planum polare, left",1284515,150,104,99,-59.0,-8.0,5.0,donor3
11302,12.635692,5.047398,1.880242,6.343823,12.001282,3.010202,8.094641,5.526588,3.600567,3.459671,...,STG-l,"superior temporal gyrus, left, lateral bank of...",1280036,158,105,99,-67.0,-8.0,4.0,donor3


In [10]:
df_all[3]

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,...,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,donor
126439333,12.668165,5.265878,1.629761,5.284850,9.320815,3.434027,9.591122,4.672892,4.361163,3.198709,...,GPi,"globus pallidus, internal segment, left",114356712,104,108,88,-13.0,3.0,1.0,donor4
126432538,12.383563,4.984746,1.902150,6.602719,11.705152,3.027106,8.300779,4.594322,3.905210,3.510353,...,HG,"Heschl's gyrus, left",1407832,144,112,97,-53.0,-6.0,-3.0,donor4
126435762,12.433869,5.256781,2.841570,6.023792,10.619858,2.962560,8.668816,4.849864,1.242969,2.848064,...,He-IV,"IV, left, lateral hemisphere",113255856,107,127,124,-16.0,-33.0,-18.0,donor4
126439285,12.604221,6.025966,1.878753,5.825703,11.514246,3.229238,8.925490,5.839518,3.116854,3.311976,...,PHG-l,"parahippocampal gyrus, left, lateral bank of g...",114357218,108,143,87,-17.0,4.0,-34.0,donor4
126435744,12.397088,5.580429,1.922846,6.643124,11.544962,3.647850,8.861175,4.964262,3.780269,2.258278,...,PHG-l,"parahippocampal gyrus, left, lateral bank of g...",111793985,114,127,117,-23.0,-26.0,-18.0,donor4
126432216,12.372845,5.146353,1.526870,6.049476,11.586096,3.185461,8.907708,5.104129,2.615426,3.285393,...,TP-m,"temporal pole, left, medial aspect",1407369,115,151,87,-24.0,4.0,-42.0,donor4
126435075,12.657814,4.888686,2.779060,7.072416,12.157647,2.862867,8.538297,4.948710,3.642408,3.362504,...,LiG-str,"lingual gyrus, left, striate",112718484,98,108,158,-7.0,-67.0,1.0,donor4
126431705,12.395018,5.136246,1.673583,6.437496,11.797971,2.962427,8.295938,7.202076,3.701269,3.423814,...,MFG-s,"middle frontal gyrus, left, superior bank of g...",1403304,120,62,65,-29.0,26.0,47.0,donor4
126434371,12.228768,5.279956,1.800253,5.669444,11.317448,3.269847,8.654623,5.187642,4.417153,3.331365,...,ITG-its,"inferior temporal gyrus, left, bank of the its",1407368,117,154,87,-26.0,4.0,-45.0,donor4
126432638,12.186885,5.287667,1.761090,5.948413,11.598550,3.080862,8.513666,5.286702,3.418303,3.364380,...,FuG-its,"fusiform gyrus, left, bank of the its",1407818,126,148,98,-35.0,-7.0,-39.0,donor4


In [11]:
df_all[4]

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,...,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,donor
141667089,12.216923,4.878327,1.351646,6.541192,12.182163,2.458788,7.840581,5.184563,3.081331,3.899325,...,SOG-i,"superior occipital gyrus, left, inferior bank ...",127562730,141,88,171,-50.0,-80.0,21.0,donor5
141667177,12.415516,5.248741,1.700870,6.462244,11.794496,2.519380,7.854143,5.331488,2.190917,3.530119,...,CgGr-s,"cingulate gyrus, retrosplenial part, left, sup...",127512019,109,111,144,-18.0,-53.0,-2.0,donor5
141667207,12.578205,5.079578,1.431586,6.579631,12.365258,2.414339,8.479937,5.179049,3.183998,3.481347,...,PoG-pcs,"postcentral gyrus, left, bank of the posterior...",127050458,138,59,118,-47.0,-27.0,50.0,donor5
139409861,12.566850,5.368312,1.351722,6.174279,12.179179,2.896632,8.494656,5.720428,2.664837,4.214436,...,SMG-i,"supramarginal gyrus, left, inferior bank of gyrus",127050410,159,79,119,-68.0,-28.0,30.0,donor5
139409845,12.524196,5.294441,2.305288,6.291404,12.227917,1.961407,8.246789,5.077249,3.165482,3.797439,...,SMG-s,"supramarginal gyrus, left, superior bank of gyrus",127050409,149,64,119,-58.0,-28.0,45.0,donor5
139409841,12.635717,5.351850,1.148410,5.890216,12.058875,3.123549,8.470478,5.592657,4.285018,3.648660,...,ITG-its,"inferior temporal gyrus, left, bank of the its",126525177,125,147,93,-34.0,-2.0,-38.0,donor5
139409825,12.420505,4.662103,1.939176,6.029179,12.249163,2.248961,8.235830,5.517764,1.618580,3.468603,...,MTG-s,"middle temporal gyrus, left, superior bank of ...",126703433,154,126,94,-63.0,-3.0,-17.0,donor5
127687007,12.579464,5.082854,1.613049,6.273017,11.574548,2.279691,8.426736,5.576212,4.010564,3.638214,...,CgGf-i,"cingulate gyrus, frontal part, left, inferior ...",123644654,95,89,71,-4.0,20.0,20.0,donor5
127687057,12.387132,5.129671,1.233883,6.072657,11.723003,2.580109,8.787872,5.320553,3.811844,3.330997,...,PLP,"planum polare, left",123636950,143,122,74,-52.0,17.0,-13.0,donor5
127687001,12.490131,5.291311,1.250447,6.418064,12.126704,2.201767,8.014572,4.888998,2.072191,3.909090,...,SRoG,"superior rostral gyrus, left",121081990,95,123,31,-4.0,60.0,-14.0,donor5


In [12]:
df_all[5]

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,...,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,donor
160535133,11.910527,4.393986,0.974908,5.037044,11.441434,3.049757,8.951266,4.993544,3.516486,2.603628,...,7,"facial motor nucleus, left",159127686,83,144,132,8.0,-41.0,-35.0,donor6
160535307,12.404726,4.289177,1.028869,4.684117,10.300250,2.804438,9.614341,5.126696,3.708341,3.289085,...,6,"abducens nucleus, left",159122217,89,143,134,2.0,-43.0,-34.0,donor6
160535291,12.567677,5.225004,1.340397,5.308471,10.072753,3.232524,9.427397,4.835249,5.282029,3.722703,...,RPn,"pontine raphe nucleus, left",158763790,92,144,132,-1.0,-41.0,-35.0,donor6
160535275,13.013984,5.496016,1.206306,6.177618,10.144302,3.361947,10.197965,5.380921,4.737649,3.909283,...,8Ve,"vestibular nuclei, left",159233515,82,144,134,9.0,-43.0,-35.0,donor6
160535259,12.574042,5.412059,1.547726,5.949662,10.814045,3.312976,8.985808,5.082649,4.651419,3.129015,...,Sp5,"spinal trigeminal nucleus, left",156687325,80,144,131,11.0,-40.0,-35.0,donor6
160535227,13.109817,4.432632,0.824527,5.860642,12.062267,3.016422,9.131485,5.199205,4.784959,3.169399,...,Pn,"pontine nuclei, left",158931290,87,145,120,4.0,-29.0,-36.0,donor6
160535211,13.096392,4.446315,1.178004,5.930496,11.178790,3.354540,9.630449,5.585004,3.582464,3.258138,...,Pr5,"principal sensory nucleus of trigeminal nerve,...",156681113,80,144,132,11.0,-41.0,-35.0,donor6
160535195,12.335754,5.058733,1.419396,4.838036,11.184425,2.954073,9.175128,5.293047,3.893330,3.365422,...,Mo5,"motor nucleus of trigeminal nerve, left",158173122,83,144,131,8.0,-40.0,-35.0,donor6
160535147,12.282708,6.181997,0.970259,5.550401,10.545507,3.305737,9.411051,4.714670,4.966648,2.904243,...,SubC,"nucleus subceruleus, left",156260971,85,144,132,6.0,-41.0,-35.0,donor6
160535131,12.771622,4.282375,1.533682,5.779639,11.075385,3.648864,9.253936,6.171168,2.837546,2.742575,...,Pr5,"principal sensory nucleus of trigeminal nerve,...",156257330,79,140,130,12.0,-39.0,-31.0,donor6


In [15]:
946+893+363+529+470+501

3702

In [16]:
AHBA = pd.concat(df_all, axis = 0)

In [17]:
MATRIX = AHBA.iloc[:,:-13]

In [18]:
for i in AHBA.columns:
    print i 
    

61E3.4
A1BG
A1CF
A26C1B
A2BP1
A2LD1
A2M
A2ML1
A3GALT2P
A4GALT
A4GNT
AAAS
AACS
AADAC
AADACL1
AADACL2
AADACL3
AADACL4
AADAT
AAED1
AAGAB
AAK1
AAMP
AANAT
AARD
AARS
AARS2
AARSD1
AASDH
AASDHPPT
AASS
AATF
AATK
ABAT
ABCA1
ABCA10
ABCA11P
ABCA12
ABCA13
ABCA17P
ABCA2
ABCA3
ABCA4
ABCA5
ABCA6
ABCA7
ABCA8
ABCA9
ABCB1
ABCB10
ABCB11
ABCB4
ABCB5
ABCB6
ABCB7
ABCB8
ABCB9
ABCC1
ABCC10
ABCC11
ABCC12
ABCC13
ABCC2
ABCC3
ABCC4
ABCC5
ABCC6
ABCC8
ABCC9
ABCD1
ABCD2
ABCD3
ABCD4
ABCE1
ABCF1
ABCF2
ABCF3
ABCG1
ABCG2
ABCG4
ABCG5
ABCG8
ABHD1
ABHD10
ABHD11
ABHD12
ABHD12B
ABHD13
ABHD14A
ABHD14B
ABHD15
ABHD2
ABHD3
ABHD4
ABHD5
ABHD6
ABHD8
ABI1
ABI2
ABI3
ABI3BP
ABL1
ABL2
ABLIM1
ABLIM2
ABLIM3
ABO
ABP1
ABR
ABRA
ABT1
ABTB1
ABTB2
AC002351.1
AC002472.1
AC002472.8
AC002472.9
AC002511.1
AC002563.2
AC003999.1
AC004053.1
AC004080.12
AC004241.1
AC004381.2
AC004409.1
AC004690.2
AC004790.2
AC004797.1
AC004863.6
AC004980.7
AC005014.1
AC005041.9
AC005077.3
AC005152.2
AC005277.1
AC005301.1
AC005358.1
AC005393.1
AC005488.13
AC005512.1
AC0

C10orf2
C10orf25
C10orf26
C10orf27
C10orf28
C10orf32
C10orf35
C10orf4
C10orf46
C10orf47
C10orf53
C10orf54
C10orf55
C10orf57
C10orf62
C10orf67
C10orf68
C10orf71
C10orf72
C10orf76
C10orf78
C10orf79
C10orf81
C10orf82
C10orf88
C10orf90
C10orf91
C10orf92
C10orf93
C10orf95
C10orf96
C10orf97
C10orf99
C11ORF36
C11orf1
C11orf10
C11orf16
C11orf17
C11orf20
C11orf21
C11orf24
C11orf30
C11orf31
C11orf34
C11orf35
C11orf40
C11orf41
C11orf42
C11orf44
C11orf45
C11orf46
C11orf48
C11orf49
C11orf51
C11orf52
C11orf53
C11orf54
C11orf57
C11orf58
C11orf59
C11orf60
C11orf61
C11orf63
C11orf65
C11orf66
C11orf67
C11orf68
C11orf70
C11orf71
C11orf73
C11orf74
C11orf75
C11orf80
C11orf82
C11orf83
C11orf84
C11orf85
C11orf86
C11orf87
C11orf88
C11orf9
C11orf92
C11orf95
C11orf96
C12ORF47
C12ORF75
C12orf10
C12orf11
C12orf12
C12orf23
C12orf24
C12orf26
C12orf28
C12orf29
C12orf30
C12orf32
C12orf33
C12orf34
C12orf35
C12orf36
C12orf37
C12orf39
C12orf4
C12orf40
C12orf41
C12orf42
C12orf43
C12orf44
C12orf45
C12orf48
C12orf49
C12orf

DENND2D
DENND3
DENND4A
DENND4B
DENND4C
DENND5A
DENND5B
DENR
DEPDC1
DEPDC1B
DEPDC4
DEPDC5
DEPDC6
DEPDC7
DERA
DERL1
DERL2
DERL3
DES
DET1
DEXI
DFFA
DFFB
DFNA5
DFNB31
DFNB59
DGAT1
DGAT2
DGAT2L6
DGCR14
DGCR2
DGCR5
DGCR6
DGCR6L
DGCR8
DGCR9
DGKA
DGKB
DGKD
DGKE
DGKG
DGKH
DGKI
DGKK
DGKQ
DGKZ
DGUOK
DHCR24
DHCR7
DHDDS
DHDH
DHFRL1
DHFRP1
DHH
DHODH
DHPS
DHRS1
DHRS11
DHRS12
DHRS13
DHRS2
DHRS3
DHRS4
DHRS4L1
DHRS4L2
DHRS7
DHRS7B
DHRS7C
DHRS9
DHRSX
DHTKD1
DHX15
DHX16
DHX29
DHX30
DHX32
DHX33
DHX34
DHX35
DHX36
DHX37
DHX38
DHX40
DHX57
DHX58
DHX8
DHX9
DIABLO
DIAPH1
DIAPH2
DIAPH3
DICER1
DICER1-AS
DIDO1
DIMT1
DIO1
DIO2
DIO3
DIO3OS
DIP2A
DIP2B
DIP2C
DIRAS1
DIRAS2
DIRAS3
DIRC1
DIRC2
DIS3
DIS3L
DIS3L2
DISC1
DISP1
DISP2
DIXDC1
DKC1
DKFZP434A062
DKFZP434H168
DKFZP434I0714
DKFZP586I1420
DKFZP779L1853
DKFZp434L192
DKFZp566H0824
DKK1
DKK2
DKK3
DKK4
DKKL1
DLAT
DLC1
DLD
DLEC1
DLEU1
DLEU2
DLEU7
DLG1
DLG2
DLG3
DLG4
DLG5
DLGAP1
DLGAP2
DLGAP3
DLGAP4
DLGAP5
DLK1
DLK2
DLL1
DLL3
DLL4
DLSTP
DLX1
DLX2
DLX3
DLX4
DLX5
DLX6
DLX6-

HMGB3P27
HMGB3P30
HMGB3P6
HMGB4
HMGCL
HMGCLL1
HMGCR
HMGCS1
HMGCS2
HMGN1
HMGN2
HMGN2P15
HMGN2P46
HMGN3
HMGN4
HMGXB3
HMGXB4
HMHA1
HMHB1
HMMR
HMOX1
HMOX2
HMSD
HMX1
HMX2
HMX3
HN1
HN1L
HNF1A
HNF1A-AS1
HNF1B
HNF4A
HNF4G
HNMT
HNRNPA0
HNRNPA1P12
HNRNPA1P2
HNRNPA2B1
HNRNPA3
HNRNPA3P1
HNRNPAB
HNRNPC
HNRNPCL1
HNRNPD
HNRNPF
HNRNPH1
HNRNPH2
HNRNPH3
HNRNPK
HNRNPKP3
HNRNPL
HNRNPM
HNRNPR
HNRNPU
HNRNPUL1
HNRNPUL2
HNRPDL
HNRPLL
HOMER1
HOMER2
HOMER3
HOMEZ
HOOK1
HOOK2
HOOK3
HOPX
HORMAD1
HORMAD2
HOTAIR
HOXA1
HOXA10
HOXA11
HOXA11-AS1
HOXA13
HOXA2
HOXA3
HOXA4
HOXA5
HOXA6
HOXA7
HOXA9
HOXB1
HOXB13
HOXB2
HOXB3
HOXB4
HOXB5
HOXB6
HOXB7
HOXB8
HOXB9
HOXC10
HOXC11
HOXC12
HOXC13
HOXC4
HOXC5
HOXC6
HOXC8
HOXC9
HOXD-AS1
HOXD1
HOXD10
HOXD11
HOXD12
HOXD13
HOXD3
HOXD4
HOXD8
HOXD9
HP1BP3
HPCA
HPCAL1
HPCAL4
HPD
HPDL
HPGD
HPN
HPR
HPRT1
HPS1
HPS3
HPS4
HPS5
HPS6
HPSE
HPSE2
HPX
HPX-2
HR
HR44
HRAS
HRASLS
HRASLS2
HRASLS5
HRC
HRCT1
HRG
HRH1
HRH2
HRH3
HRH4
HRK
HRNR
HRSP12
HS1BP3
HS2ST1
HS3ST1
HS3ST2
HS3ST3A1
HS3ST3B1
HS3ST4
HS3ST5
H

LOC286126
LOC286238
LOC286359
LOC286434
LOC338620
LOC338667
LOC338756
LOC339240
LOC339290
LOC339316
LOC339352
LOC339524
LOC339760
LOC340085
LOC340888
LOC340947
LOC341056
LOC341378
LOC342541
LOC343052
LOC344382
LOC344593
LOC346329
LOC375196
LOC375295
LOC377711
LOC387820
LOC388237
LOC388248
LOC388630
LOC388692
LOC388849
LOC388955
LOC389043
LOC389199
LOC389332
LOC389458
LOC389607
LOC389705
LOC389765
LOC389787
LOC389791
LOC389831
LOC389834
LOC389842
LOC389895
LOC390282
LOC390660
LOC390705
LOC390760
LOC391322
LOC391334
LOC391722
LOC391798
LOC392145
LOC392264
LOC392288
LOC392352
LOC392555
LOC393078
LOC399744
LOC399753
LOC399815
LOC399900
LOC399978
LOC400027
LOC400099
LOC400352
LOC400406
LOC400419
LOC400590
LOC400643
LOC400657
LOC400684
LOC400750
LOC400752
LOC400943
LOC400987
LOC401021
LOC401052
LOC401093
LOC401098
LOC401131
LOC401321
LOC401357
LOC401433
LOC401480
LOC401533
LOC401777
LOC401805
LOC401847
LOC401859
LOC401913
LOC402360
LOC402377
LOC402644
LOC407835
LOC414300
LOC415056
LOC439914


OR2A4
OR2A5
OR2A9P
OR2AE1
OR2AG1
OR2AG2
OR2AK2
OR2AT4
OR2B11
OR2B2
OR2B3
OR2C1
OR2C3
OR2D2
OR2D3
OR2F1
OR2F2
OR2G2
OR2G3
OR2G6
OR2H1
OR2H2
OR2J2
OR2J3
OR2K2
OR2L13
OR2L2
OR2L3
OR2L8
OR2M1P
OR2M2
OR2M3
OR2M4
OR2M5
OR2M7
OR2S2
OR2T1
OR2T10
OR2T11
OR2T12
OR2T2
OR2T27
OR2T33
OR2T34
OR2T4
OR2T5
OR2T6
OR2T8
OR2V2
OR2W1
OR2W3
OR2W5
OR2Y1
OR2Z1
OR3A1
OR3A2
OR3A3
OR4A15
OR4A16
OR4A5
OR4B1
OR4C11
OR4C12
OR4C13
OR4C15
OR4C16
OR4C3
OR4C45
OR4C46
OR4C6
OR4D1
OR4D10
OR4D11
OR4D2
OR4D5
OR4D6
OR4D9
OR4E2
OR4F15
OR4F21
OR4F3
OR4F5
OR4F6
OR4K1
OR4K13
OR4K14
OR4K15
OR4K17
OR4K2
OR4K5
OR4L1
OR4M1
OR4M2
OR4N2
OR4N4
OR4N5
OR4P4
OR4Q3
OR4S1
OR4S2
OR4X1
OR4X2
OR51A2
OR51A4
OR51A7
OR51B2
OR51B4
OR51B5
OR51B6
OR51D1
OR51E1
OR51E2
OR51F1
OR51F2
OR51G1
OR51G2
OR51I1
OR51I2
OR51L1
OR51M1
OR51Q1
OR51S1
OR51T1
OR51V1
OR52A1
OR52A4
OR52A5
OR52B2
OR52B4
OR52B6
OR52D1
OR52E2
OR52E4
OR52E6
OR52E8
OR52I1
OR52I2
OR52J3
OR52K1
OR52K2
OR52L1
OR52M1
OR52N1
OR52N2
OR52N4
OR52N5
OR52R1
OR52V1P
OR52W1
OR56A1
OR56A3
OR56A4
OR56A

S100A7L2
S100A8
S100A9
S100B
S100G
S100P
S100PBP
S100Z
S1PR1
S1PR2
S1PR3
S1PR4
S1PR5
SAA1
SAA2
SAA3P
SAA4
SAAL1
SAC3D1
SACM1L
SACS
SAE1
SAFB
SAFB2
SAG
SAGE1
SALL1
SALL2
SALL3
SALL4
SAMD1
SAMD10
SAMD11
SAMD12
SAMD13
SAMD14
SAMD15
SAMD3
SAMD4A
SAMD4B
SAMD5
SAMD7
SAMD8
SAMD9
SAMD9L
SAMHD1
SAMM50
SAMSN1
SAP130
SAP18
SAP30
SAP30BP
SAP30L
SAPS1
SAPS2
SAPS3
SAR1A
SAR1B
SARDH
SARM1
SARNP
SARS
SART1
SART3
SASH1
SASH3
SASS6
SAT1
SAT2
SATB1
SATB2
SATL1
SAV1
SBDS
SBDSP1
SBF1
SBF1P1
SBF2
SBK1
SBK2
SBNO1
SBNO2
SBSN
SC4MOL
SC5DL
SCAF1
SCAF11
SCAI
SCAMP1
SCAMP2
SCAMP3
SCAMP4
SCAMP5
SCAMPER
SCAND1
SCAND2
SCAND3
SCAP
SCAPER
SCARA3
SCARA5
SCARB1
SCARB2
SCARF1
SCARF2
SCARNA17
SCCPDH
SCD
SCD5
SCEL
SCFD1
SCFD2
SCG2
SCG3
SCG5
SCGB1A1
SCGB1C1
SCGB1D1
SCGB1D2
SCGB1D4
SCGB2A1
SCGB2A2
SCGB3A1
SCGB3A2
SCGBL
SCGN
SCIMP
SCIN
SCLT1
SCLY
SCMH1
SCML1
SCML2
SCML4
SCN10A
SCN11A
SCN1A
SCN1B
SCN2A
SCN2B
SCN3A
SCN3B
SCN4A
SCN4B
SCN5A
SCN7A
SCN8A
SCN9A
SCNM1
SCNN1A
SCNN1B
SCNN1D
SCNN1G
SCO1
SCO2
SCOC
SCP2
SCPEP1
SCRG1
SCRIB

TSPO
TSPY1
TSPY26P
TSPYL1
TSPYL2
TSPYL5
TSPYL6
TSR1
TSR2
TSSC1
TSSC4
TSSK1B
TSSK2
TSSK3
TSSK4
TSSK6
TST
TSTA3
TTBK1
TTBK2
TTC1
TTC12
TTC13
TTC14
TTC15
TTC16
TTC17
TTC18
TTC19
TTC21A
TTC21B
TTC22
TTC23
TTC23L
TTC24
TTC25
TTC26
TTC27
TTC28
TTC29
TTC3
TTC30A
TTC30B
TTC31
TTC32
TTC33
TTC35
TTC36
TTC37
TTC38
TTC39A
TTC39B
TTC39C
TTC4
TTC5
TTC7A
TTC7B
TTC8
TTC9
TTC9B
TTC9C
TTF1
TTF2
TTK
TTL
TTLL1
TTLL10
TTLL11
TTLL12
TTLL13
TTLL2
TTLL3
TTLL4
TTLL5
TTLL6
TTLL7
TTLL8
TTLL9
TTN
TTPA
TTPAL
TTR
TTRAP
TTTY1
TTTY10
TTTY11
TTTY12
TTTY13
TTTY14
TTTY15
TTTY2
TTTY20
TTTY21
TTTY22
TTTY23
TTTY3B
TTTY4B
TTTY5
TTTY6B
TTTY7
TTTY8
TTTY9B
TTYH1
TTYH2
TTYH3
TUB
TUBA1A
TUBA1B
TUBA1C
TUBA3C
TUBA3D
TUBA3E
TUBA4A
TUBA4B
TUBA8
TUBAL3
TUBAP2
TUBB
TUBB1
TUBB2A
TUBB2B
TUBB2C
TUBB4
TUBB7P
TUBB8
TUBD1
TUBE1
TUBG1
TUBG2
TUBGCP2
TUBGCP3
TUBGCP4
TUBGCP5
TUBGCP6
TUFM
TUFT1
TUG1
TULP1
TULP2
TULP3
TULP4
TUSC1
TUSC2
TUSC3
TUSC4
TUSC5
TUT1
TWF1
TWF2
TWIST1
TWIST2
TWISTNB
TWSG1
TXK
TXLNA
TXLNB
TXN
TXN2
TXNDC11
TXNDC12
TXNDC15
TX

In [22]:
AHBA.to_csv(os.path.join(datadir,'AHBA_20737.csv'))

In [20]:
MATRIX.head()

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,na
594,12.257452,4.307083,1.639045,6.047821,11.502535,2.992096,7.702302,4.657058,1.675294,3.470479,...,2.459725,5.819813,4.370754,5.060327,1.842454,7.411024,6.758185,4.880300,5.840162,3.886859
2985,12.971966,5.394092,2.016619,6.258501,12.314500,3.450627,8.285363,5.044534,3.596557,4.651128,...,2.977191,7.124811,5.075309,5.837238,2.831383,8.172978,8.090333,5.372033,7.369528,4.368570
2801,13.397286,6.196477,2.419608,5.442959,12.776440,3.652065,8.698773,4.777458,3.662503,4.845015,...,4.847570,5.714349,4.428118,5.517012,3.062045,7.921725,6.913591,5.248995,7.393073,4.762620
2273,13.326607,5.665946,2.473735,4.363560,12.428296,4.512593,9.333068,5.325190,2.956374,3.377773,...,2.800116,4.058270,4.039316,5.697372,2.232414,6.656573,7.242092,5.496471,5.060012,4.675658
2785,13.356659,5.087975,2.391748,4.590116,13.365749,3.558010,9.939992,5.247008,3.760852,4.446860,...,3.842781,6.337220,4.642517,5.773350,2.225465,7.344605,6.806393,5.397969,7.830510,4.720244
